# Lib Import

In [54]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Dropout, Embedding, Input, concatenate, Conv2D, BatchNormalization, Activation, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras import Model

# Modeling

In [55]:
input_a = Input(shape=[1], name="input_a")
input_b = Input(shape=[1], name="input_b")
hidden_1 = Dense(30, activation="relu")(input_b)
hidden_2 = Dense(30, activation="relu")(hidden_1)
aux_output = Dense(1, name="aux_output")(hidden_2)
concat = concatenate([input_a, hidden_2])
output = Dense(1, name="output")(concat)
model = tf.keras.Model(inputs=[input_a, input_b], outputs=[output, aux_output])

# Making Residual Model with Class

In [56]:
class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = Dense(units, activation=activation)
        self.hidden2 = Dense(units, activation=activation)
        self.main_output = Dense(1)
        self.aux_output = Dense(1)

    def call(self, inputs):
        input_a, input_b = inputs
        hidden1 = self.hidden1(input_b)
        hidden2 = self.hidden2(hidden1)
        concat = concatenate([input_a, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

In [57]:
class CNNResidualLayer(Layer):
    def __init__(self, layers, filters, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [Conv2D(filters, 3, activation="relu") for _ in range(layers)]
        
    def call(self, inputs):
        x = inputs
        for layer in self.hidden:
            x = layer(x)
        return x + inputs

In [58]:
class DNNResidualLayer(Layer):
    def __init__(self,layers, units, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [Dense(units, activation="relu") for _ in range(layers)]
        
    def call(self, inputs):
        x = inputs
        for layer in self.hidden:
            x = layer(x)
        return x + inputs

In [59]:
class CustomResidual(Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.block1 = CNNResidualLayer(2, 32)
        self.block2 = DNNResidualLayer(2, 64)
        self.out = Dense(1)
        
    def call(self, inputs):
        x = self.block1(inputs)
        x = self.block2(x)
        for _ in range(1, 3):
            x = self.block2(x)
        return self.out(x)

# Mini Resnet

In [78]:
class IdentityBlock(Model):
    def __init__(self, filters, kernel_size):
        super(IdentityBlock, self).__init__(name='')	
        self.conv1 = Conv2D(filters, kernel_size, activation="relu", padding='same')
        self.conv2 = Conv2D(filters, kernel_size, activation="relu", padding='same')
        
        self.bn1 = BatchNormalization()
        self.bn2 = BatchNormalization()
        
        self.act = tf.keras.layers.Activation("relu")
        self.add = tf.keras.layers.Add()
        
    def call(self, input_tensor):
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        x = self.act(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.add([x, input_tensor])
        x = self.act(x)
        return x
    
class ResNet(Model):
    def __init__(self, num_classes,):
        super(ResNet, self).__init__()
        self.conv = Conv2D(64, 7, padding="same")
        self.bn = BatchNormalization()
        self.act = Activation("relu")
        self.max_pool = MaxPool2D((3, 3))
        self.id1b = IdentityBlock(64, 3)
        self.id1a = IdentityBlock(64, 3)
        self.global_pool = GlobalAveragePooling2D()
        self.classifier = Dense(num_classes, activation="softmax")
        
    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.act(x)
        x = self.max_pool(x)
        x = self.id1b(x)
        x = self.id1a(x)
        x = self.global_pool(x)
        return self.classifier(x)
        

In [79]:
from tensorflow.keras.datasets import mnist 
import tensorflow_datasets as tfds

def preprocess(features):
    return tf.cast(features['image'], tf.float32) / 255., features['label']

resnet = ResNet(10)
resnet.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
dataset = tfds.load("mnist", split="train")
dataset = dataset.map(preprocess).batch(32)
resnet.fit(dataset, epochs=3)

Epoch 1/3
1875/1875 [==============================] - 145s 75ms/step - loss: 0.1152 - accuracy: 0.9701
Epoch 2/3
1875/1875 [==============================] - 142s 76ms/step - loss: 0.0428 - accuracy: 0.9873
Epoch 3/3
1875/1875 [==============================] - 134s 71ms/step - loss: 0.0306 - accuracy: 0.9911
